# Testzone
Script tester zone. Used to try code and ideas.

# Train and test a Scikit-learn Classifier
In this example, the `make_classification` function is used to generate some synthetic data for the example, but in a real-world scenario, you would use your own data. The data is first split into training and test sets using the `train_test_split` function. The `RandomForestClassifier` is initialized with a random state. The classifier is then fit to the training data using the fit method.

In [1]:
# Simple example of how to train a model and test it.
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# generate some synthetic data for the example
X, y = make_classification(n_samples=1000, n_features=4, n_classes=2, random_state=0)

# display the first 5 rows of the data
print("Sample of feature vectors X:", X[:5], "and associated labels y:", y[:5], sep="\n")

# split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# initialize the classifier[:10]
clf = RandomForestClassifier(random_state=0)

# fit the classifier to the training data
clf.fit(X_train, y_train)

Sample of feature vectors X:
[[ 1.27815198 -0.41644753  0.89181112  0.77129444]
 [ 1.35681817 -1.51465569  1.82132242  0.42081175]
 [ 1.53341056  2.06290707 -1.01967188  1.87609016]
 [ 0.42064934  0.05455201  0.13725671  0.32493018]
 [-0.88825673 -1.10088618  0.51393811 -1.05185003]]
and associated labels y:
[1 1 1 1 0]


RandomForestClassifier(random_state=0)

The `predict` method is then used to make predictions on the test data, and the `accuracy_score` function is used to calculate the accuracy of the model. The `classification_report` function is also used to generate a more detailed report of the model's performance, including precision, recall, and F1-score for each class.

In [2]:
from sklearn.metrics import accuracy_score, classification_report

# make predictions on the test data
y_pred = clf.predict(X_test)
print(
    "Sample of predicted labels:", y_pred[:20], 
    "versus actual labels:", y_test[:20], sep="\n"
)

# calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

# print the classification report
print(classification_report(y_test, y_pred))

Sample of predicted labels:
[1 0 0 0 0 1 1 1 1 1 0 0 1 1 1 1 1 1 0 0]
versus actual labels:
[1 0 0 0 0 1 1 1 1 1 0 1 0 1 0 1 1 1 0 0]
Accuracy: 97.00%
              precision    recall  f1-score   support

           0       0.98      0.96      0.97        95
           1       0.96      0.98      0.97       105

    accuracy                           0.97       200
   macro avg       0.97      0.97      0.97       200
weighted avg       0.97      0.97      0.97       200



Here, the `confusion_matrix` function is used to calculate a confusion matrix, which is a 2x2 table that shows the number of true positives, false positives, true negatives, and false negatives. The `roc_curve` function is used to calculate the false positive rate and true positive rate, which are used to plot the ROC curve. The `auc` function is used to calculate the area under the ROC curve, which is a measure of the classifier's performance. Finally, the ROC curve is plotted using `matplotlib`.

In [3]:
from sklearn.metrics import confusion_matrix, roc_curve, auc

# calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix: \n", cm)
print("True Positives: ", cm[1, 1])
print("True Negatives: ", cm[0, 0])
print("False Positives: ", cm[0, 1])
print("False Negatives: ", cm[1, 0])

# calculate the false positive rate and true positive rate
fpr, tpr, thresholds = roc_curve(y_test, y_pred)

# calculate the area under the ROC curve
roc_auc = auc(fpr, tpr)
print("AUC: {:.2f}".format(roc_auc))

# plot the ROC curve
import matplotlib.pyplot as plt
plt.plot(fpr, tpr, label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate or (1 - Specifity)')
plt.ylabel('True Positive Rate or (Sensitivity)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

Confusion Matrix: 
 [[ 91   4]
 [  2 103]]
True Positives:  103
True Negatives:  91
False Positives:  4
False Negatives:  2
AUC: 0.97


ModuleNotFoundError: No module named 'matplotlib'

## Training and testing a different classifier


In [6]:
import utils
import os
import time
import pandas as pd
import numpy as np
from datetime import datetime
import pickle

# machine learning
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from classes import *
import utils
import train_utils

In [2]:
# program parameters
from dataclasses import dataclass

@dataclass
class ProgramParameters:
    """
    Wrapper class for program parameters.
    """
    MODEL_DIR_PATH = os.environ['HOME'] + "/Documents/code/phdtrack/Smart-and-Naive-SSH-Key-Extraction/smart_ssh_key_extractor/new_models"
    TRAINING_DIR_PATH = os.environ['HOME'] + '/Documents/code/phdtrack/phdtrack_data/Training/Training/scp/'
    VALIDATION_DIR_PATH = os.environ['HOME'] + '/Documents/code/phdtrack/phdtrack_data/Validation/Validation/scp/V_7_8_P1/16'
    RESULTS_PATH = os.environ['HOME'] + "/Documents/code/phdtrack/Smart-and-Naive-SSH-Key-Extraction/smart_ssh_key_extractor/results"
    

    def __init__(self, **kwargs):
        if (
            utils.check_path_exists(self.MODEL_DIR_PATH) and
            utils.check_path_exists(self.TRAINING_DIR_PATH) and
            utils.check_path_exists(self.VALIDATION_DIR_PATH) and
            utils.check_path_exists(self.RESULTS_PATH)
        ):
            print("Program paths are OK.")
        else:
            print("Program paths are NOT OK.")
            exit(1)


PARAMS = ProgramParameters()

Program paths are OK.


In [3]:
# get the data, create datasets
file_paths, key_paths = utils.get_dataset_file_paths(PARAMS.TRAINING_DIR_PATH)
print("Number of raw files: ", len(file_paths))
print("Number of keys json files: ", len(key_paths))

all_block_datas, nb_invalid_blocks = train_utils.create_dataset(PARAMS.TRAINING_DIR_PATH)
print("Number of block datas: ", len(all_block_datas))
print("Number of invalid blocks (ignored): ", nb_invalid_blocks)

# count and remove the invalid block datas
invalid_block_datas = 0
for block_data in all_block_datas:
    if block_data.label == -1:
        invalid_block_datas += 1


# transform the data into lists of samples and labels
X = []
y = []
for block_data in all_block_datas:
    sample = block_data.dataset
    label = block_data.label

    X.append(sample)
    y.append(label)

print("Number of features: ", len(X[0]))
print("Number of samples: ", len(X))
print("Number of labels: ", len(y))


Number of raw files:  8501
Number of keys json files:  8501
Number of .raw files to load: 8501
Number of block datas:  23290878
Number of invalid blocks (ignored):  1737
Number of features:  128
Number of samples:  23290878
Number of labels:  23290878


In [4]:
# print the number of valid and empty block datas
valid_block_datas = 0
empty_block_datas = 0

for block_data in all_block_datas:
    if block_data.label == BlockType.VALID.value:
        valid_block_datas += 1
    elif block_data.label == BlockType.EMPTY.value:
        empty_block_datas += 1
    else:
        print("label: ", block_data.label)

print("Number of valid block datas: ", valid_block_datas)
print("Number of empty block datas: ", empty_block_datas)

Number of valid block datas:  78297
Number of empty block datas:  23212581


In [5]:
# resample the data

# Use SMOTE oversampling
USE_SMOTE_OVERSAMPLING = False

if USE_SMOTE_OVERSAMPLING:
    print("Using SMOTE oversampling")
    start = time.time()
    sm = SMOTE()
    x_train, y_train = sm.fit_resample(X, y)
    end = time.time()
    print('Time taken for resampling: %f' % (end - start))

# undersampling using RandomUnderSampler
USE_UNDERSAMPLING = True

if USE_UNDERSAMPLING:
    print("Using undersampling")
    start = time.time()
    rus = RandomUnderSampler()
    x_train, y_train = rus.fit_resample(X, y)
    end = time.time()
    print('Time taken for undersampling: %f' % (end - start))

Using undersampling
Time taken for undersampling: 43.215010


In [7]:
# print info about the training data
print("Number of training data <TRAINING>:", len(x_train))
print("Number of training labels <TRAINING>:", len(y_train))

# print the number of valid and empty block datas
valid_block_datas = 0
empty_block_datas = 0

for i in range(len(y_train)):
    if y_train[i] == BlockType.VALID.value:
        valid_block_datas += 1
    elif y_train[i] == BlockType.EMPTY.value:
        empty_block_datas += 1
    else:
        print("Big problem here: ", y_train[i])

print("Number of valid block datas <TRAINING>:", valid_block_datas)
print("Number of empty block datas <TRAINING>:", empty_block_datas)

Number of training data <TRAINING>: 156594
Number of training labels <TRAINING>: 156594
Number of valid block datas <TRAINING>: 78297
Number of empty block datas <TRAINING>: 78297


In [9]:
# train a new classifier on the resampled data
RANDOM_FOREST = False
BAGGING = False
SVC_CLF = True

if BAGGING:
    start = time.time()
    custom_clf = BaggingClassifier(estimator=RandomForestClassifier(n_estimators=5), n_estimators=3, n_jobs=8, random_state=0)
    custom_clf.fit(X=np.array(x_train), y=y_train)
    end = time.time()
    print('Time taken for training the classifier on resampled data: %f' % (end - start))
elif RANDOM_FOREST:
    start = time.time()
    custom_clf = RandomForestClassifier(n_estimators=5, n_jobs=8, random_state=0)
    custom_clf.fit(X=np.array(x_train), y=y_train)
    end = time.time()
    print('Time taken for training the classifier on resampled data: %f' % (end - start))
elif SVC_CLF:
    start = time.time()

    # set the parameter grid
    param_grid = {
        'C': [0.1, 1, 10], 
        'kernel': ['rbf'],
        'class_weight': {0: 1, 1: 100}
    }

    # initialize the classifier
    svc = SVC()

    # initialize the grid search #TODO: RandomizedSearchCV
    grid_search = GridSearchCV(svc, param_grid, cv=5, scoring='recall', n_jobs=8)

    # fit the grid search to the data
    grid_search.fit(x_train, y_train)

    # print the best parameters
    print("Best parameters: ", grid_search.best_params_)

    #get the best model
    custom_clf = grid_search.best_estimator_

    end = time.time()
    print('Time taken for training the classifier on resampled data: %f' % (end - start))

Best parameters:  {'C': 0.1, 'class_weight': {0: 1, 1: 100}, 'kernel': 'rbf'}
Time taken for training the classifier on resampled data: 16110.192132


In [10]:
# save the classifier
model_file_path = os.path.join(PARAMS.MODEL_DIR_PATH, "new_classifier_svc_1.pkl")
with open(model_file_path, 'wb') as fp:
    pickle.dump(custom_clf, fp)

In [11]:
# load the classifier
model_file_path = os.path.join(PARAMS.MODEL_DIR_PATH, "new_classifier_svc_1.pkl")
with open(model_file_path, 'rb') as fp:
    custom_clf = pickle.load(fp)

In [12]:
# testing model on the test data
test_file_paths, test_key_paths = utils.get_dataset_file_paths(PARAMS.VALIDATION_DIR_PATH)
print("Number of test raw files: ", len(test_file_paths))
print("Number of test keys json files: ", len(test_key_paths))

print('Testing Dataset')
start = time.time()
y_test, y_pred, df = utils.test(
    clf=custom_clf, 
    file_paths=test_file_paths, 
    key_paths=test_key_paths
)
end = time.time()
print('Time taken for reading and testing: %f' % (end - start))

path = os.path.join(PARAMS.RESULTS_PATH, "test_results_" + str(datetime.now()) + ".csv")
df.to_csv(path)

start = time.time()
print('METRICS OF TEST SET')
utils.print_metrics(y_test=y_test, y_pred=y_pred)
end = time.time()
print('Time taken for computing metrics: %f' % (end - start))

Number of test raw files:  181
Number of test keys json files:  181
Testing Dataset
['ode', 'phdtrack', 'phdtrack_data', 'Validation', 'Validation', 'scp', 'V_7_8_P1', '16']
Time taken for reading and testing: 1224.978734
METRICS OF TEST SET
2023-01-16 01:35:52.298346:	Accuracy: 0.596368
2023-01-16 01:35:52.298399:	Precision: 0.016778
2023-01-16 01:35:52.298406:	Recall: 1.000000
2023-01-16 01:35:52.298411:	F1-Measure: 0.033001
2023-01-16 01:35:52.298499:	
Confusion Matrix:
[[226976 155416]
 [     0   2652]]
2023-01-16 01:35:52.298508:	True Positives: 2652
2023-01-16 01:35:52.298514:	True Negatives: 226976
2023-01-16 01:35:52.298520:	False Positives: 155416
2023-01-16 01:35:52.298525:	False Negatives: 0
Time taken for computing metrics: 1.069762
